### Cargar los DataFrames limpios

In [1]:
import pandas as pd

df_netflix = pd.read_parquet("netflix_limpio.parquet")

df_amazon = pd.read_parquet("amazon_limpio.parquet")

df_disney = pd.read_parquet("disney_limpio.parquet")

### Concatenar los DataFrames 

In [2]:
df_disney["plataforma"] = "Disney"
df_netflix["plataforma"] = "Netflix"
df_amazon["plataforma"] = "Amazon"

df_streaming = pd.concat([df_disney, df_netflix, df_amazon], ignore_index=True)

In [ ]:
df_streaming.to_parquet("df_streaming_completo.parquet")

In [3]:
df_streaming.tail()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added,plataforma
10303,s9538,tv show,Pan y Circo,Desconocido,Desconocido,desconocido,2021-10-08,2021,7+,2 Seasons,"special interest, talk show and variety, unscr...",Mexican actor and director Diego Luna returns ...,2021,10,Amazon
10304,s9547,tv show,Hidden Villages,Desconocido,Penelope Keith,desconocido,2021-10-08,2016,all,2 Seasons,"documentary, special interest","In this series, renowned actress Dame Penelope...",2021,10,Amazon
10305,s9636,tv show,Pokémon the Series: Diamond and Pearl,Desconocido,Ikue Ohtani,desconocido,2021-10-10,2011,all,12 Seasons,"animation, kids","Ash, Dawn, and Brock continue to travel across...",2021,10,Amazon
10306,s9643,tv show,Lunkerville,Desconocido,Michael de Avila,desconocido,2021-10-10,2021,13+,2 Seasons,"documentary, special interest, sports","Lunkerville's launches ""Get Folks Fishing"", a ...",2021,10,Amazon
10307,s9648,tv show,Kids In The Hall,Desconocido,Desconocido,desconocido,2021-10-10,1995,tv-14,5 Seasons,"arts, entertainment, and culture, comedy","""The Kids in the Hall"" is a unique and bizarre...",2021,10,Amazon


In [4]:
df_streaming["plataforma"].value_counts()

plataforma
Netflix    8708
Disney     1445
Amazon      155
Name: count, dtype: int64

## Análisis

### Top 5 géneros más populares por plataforma

In [ ]:
# Asegura que 'listed_in' sea una lista
df_streaming["listed_in"] = df_streaming["listed_in"].apply(
    lambda x: [i.strip() for i in x.split(",")] if isinstance(x, str) else x
)

# Explota los géneros
df_genero = df_streaming.explode("listed_in")
df_genero['listed_in'] = df_genero['listed_in'].str.strip()

# Agrupa por plataforma y género
genero_plataforma = df_genero.groupby(["plataforma", "listed_in"]).size().reset_index(name="conteo")

genero_plataforma = genero_plataforma.sort_values(by=["plataforma", "conteo"], ascending=[True, False])

# Mostrar los 5 géneros más populares de cada plataforma
top5_por_plataforma = genero_plataforma.groupby("plataforma").head(5).reset_index(drop=True)
top5_por_plataforma 

In [30]:
top5_por_plataforma.to_parquet("top5_por_plataforma.parquet")

### ¿Cuántas películas y series se añaden a la plataforma cada año?

In [ ]:
df_anual = df_streaming.groupby(["year_added", "type","plataforma"]).size().reset_index(name="conteo")
df_anual.tail()

In [31]:
df_anual.to_parquet("df_anual.parquet")

### Top 5 años más productivos para cada plataforma

In [38]:
# Agrupa y ordena por plataforma y conteo descendente
df_productivos = df_streaming.groupby(["plataforma", "year_added"]).size().reset_index(name="conteo")
df_productivos = df_productivos.sort_values(["plataforma", "conteo"], ascending=[True, False])

# Selecciona el top 5 años más productivos para cada plataforma
top5_productivos = df_productivos.groupby("plataforma").head(5).reset_index(drop=True)

top5_productivos

,plataforma,year_added,conteo
0,Amazon,2021,155
1,Disney,2019,765
2,Disney,2020,352
3,Disney,2021,328
4,Netflix,2019,1999
5,Netflix,2020,1878
6,Netflix,2018,1625
7,Netflix,2021,1498
8,Netflix,2017,1163


In [39]:
top5_productivos.to_parquet("top5_productivos.parquet")

In [43]:
import pandas as pd

df = pd.read_parquet("df_streaming_completo.parquet")
anio_mas_antiguo_amazon = df[df["plataforma"] == "Amazon"]["year_added"].max()
print("El año más reciente en que se añadió un contenido en Amazon es:", anio_mas_antiguo_amazon)

El año más reciente en que se añadió un contenido en Amazon es: 2021


In [44]:
import pandas as pd

df = pd.read_parquet("df_streaming_completo.parquet")
anio_mas_antiguo_amazon = df[df["plataforma"] == "Amazon"]["year_added"].min()
print("El año más antiguo en que se añadió un contenido en Amazon es:", anio_mas_antiguo_amazon)

El año más antiguo en que se añadió un contenido en Amazon es: 2021


### ¿Son más populares las películas o series en los últimos 5 años?

In [45]:
# Filtra los últimos 5 años
df_reciente = df_streaming[df_streaming["year_added"] >= 2018]

# Agrupa por plataforma y tipo
popularidad_reciente = (
    df_reciente.groupby(["plataforma", "type"])
    .size()
    .reset_index(name="conteo")
    .sort_values(["plataforma", "conteo"], ascending=[True, False])
)

popularidad_reciente

,plataforma,type,conteo
1,Amazon,tv show,139
0,Amazon,movie,16
2,Disney,movie,1052
3,Disney,tv show,393
4,Netflix,movie,4938
5,Netflix,tv show,2062


In [46]:
popularidad_reciente.to_parquet("popularidad_reciente.parquet")

### Top 5 países con más contenido por plataforma

In [48]:
# Asegura de que country sea lista
df_streaming["country"] = df_streaming["country"].apply(
    lambda x: [i.strip() for i in x.split(",")] if isinstance(x, str) else x
)

# Explota la columna country
df_paises = df_streaming.explode("country")
df_paises["country"] = df_paises["country"].str.strip()

# Agrupa por plataforma y país, cuenta títulos
conteo_paises = df_paises.groupby(["plataforma", "country"]).size().reset_index(name="conteo")

# Ordena y selecciona el top 5 países por plataforma
top5_paises_por_plataforma = (
    conteo_paises.sort_values(["plataforma", "conteo"], ascending=[True, False])
    .groupby("plataforma")
    .head(5)
    .reset_index(drop=True)
)
top5_paises_por_plataforma

,plataforma,country,conteo
0,Amazon,desconocido,129
1,Amazon,united states,14
2,Amazon,canada,3
3,Amazon,spain,3
4,Amazon,united kingdom,3
5,Disney,united states,1182
6,Disney,desconocido,217
7,Disney,united kingdom,101
8,Disney,canada,76
9,Disney,australia,23


In [49]:
top5_paises_por_plataforma.to_parquet("top5_paises_por_plataforma.parquet")

### Top 5 rating del contenido por plataforma

In [50]:
# Agrupa por plataforma y clasificación, cuenta títulos
df_rating_plat = df_streaming.groupby(["plataforma", "rating"]).size().reset_index(name="conteo")

# Ordena y selecciona el top 5 clasificaciones por plataforma
top5_rating_por_plataforma = (
    df_rating_plat.sort_values(["plataforma", "conteo"], ascending=[True, False])
    .groupby("plataforma")
    .head(5)
    .reset_index(drop=True)
)
top5_rating_por_plataforma

,plataforma,rating,conteo
0,Amazon,all,35
1,Amazon,13+,20
2,Amazon,16+,15
3,Amazon,desconocido,15
4,Amazon,tv-g,13
5,Disney,tv-g,318
6,Disney,tv-pg,301
7,Disney,g,253
8,Disney,pg,236
9,Disney,tv-y7,129


In [51]:
top5_rating_por_plataforma.to_parquet("top5_rating_por_plataforma.parquet")

### Top 5 clasificaciones por tipo de contenido

In [52]:
# Agrupa por tipo y clasificación, cuenta títulos
df_rating_tipo = df_streaming.groupby(["type", "rating"]).size().reset_index(name="conteo")

# Ordena y selecciona el top 5 clasificaciones por tipo de contenido
top5_rating_por_tipo = (
    df_rating_tipo.sort_values(["type", "conteo"], ascending=[True, False])
    .groupby("type")
    .head(5)
    .reset_index(drop=True)
)
top5_rating_por_tipo

,type,rating,conteo
0,movie,tv-ma,2062
1,movie,tv-14,1464
2,movie,r,797
3,movie,tv-pg,721
4,movie,pg-13,556
5,tv show,tv-ma,1126
6,tv show,tv-14,758
7,tv show,tv-pg,422
8,tv show,tv-y7,289
9,tv show,tv-y,225


In [53]:
top5_rating_por_tipo.to_parquet("top5_rating_por_tipo.parquet")